In [20]:
# pip install webdriver_manager --trusted-host pypi.org --trusted-host files.pythonhosted.org
# pip install certifi --trusted-host pypi.org --trusted-host files.pythonhosted.org
# pip install --upgrade tabulate --trusted-host pypi.org --trusted-host files.pythonhosted.org
# pip install openpyxl --trusted-host pypi.org --trusted-host files.pythonhosted.org

import pandas as pd
import requests
import ssl
import xml.etree.ElementTree as ET
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from datetime import date

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [21]:
###########################################
### 자동차 산업 (출처 : 차동차 산업협회)
### MOTOR_PROD_QUAN : 자동차산업동향-생산수량(대)
### MOTOR_DOME_QUAN : 자동차산업동향-내수판매량(대)
### MOTOR_EXPORT_QUAN : 자동차산업동향-수출수량(대)
### MOTOR_EXPORT_SALES : 자동차산업동향-수출금액(천 달러)
###########################################

today = date.today()

url = "https://www.kama.or.kr/MainController" 

def motorCrawlData(driver):
    driver.get(url)
    driver.find_element(By.XPATH, '//*[@id="gnb"]/li[1]/a[1]').click()
    time.sleep(5)
    driver.find_element(By.XPATH, '//*[@id="user_id"]').send_keys("kcredit")
    driver.find_element(By.XPATH, '//*[@id="user_pw"]').send_keys("kcredit2023!")
    driver.find_element(By.XPATH, '//*[@id="content"]/form/div/div/a').send_keys(Keys.ENTER)
    time.sleep(5)

    # 자동차통계DB(STATISTICS) 클릭
    driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul[2]/li[1]/a').send_keys(Keys.ENTER)
    time.sleep(5)

    # 조회조건 설정 : 2020년 1월 부터 최근년도 12월까지
    select = Select(driver.find_element(By.XPATH, '//*[@id="statsYear"]'))
    select.select_by_visible_text('2010년')
    select.select_by_value('2010')
    select = Select(driver.find_element(By.XPATH, '//*[@id="statsMonthE"]'))
    select.select_by_visible_text('12월')
    select.select_by_value('12')
    driver.find_element(By.XPATH, '//*[@id="checkBox"]/table/tbody/tr[2]/td/label').click()
    driver.find_element(By.XPATH, '//*[@id="checkBox"]/div/a').send_keys(Keys.ENTER)

    print(driver.window_handles)
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(5)

    # Printing the whole body text
    # print(browser.find_element(By.XPATH, "/html/body").text)
    driver.find_element(By.XPATH, '//*[@id="proceed-button"]').send_keys(Keys.ENTER)
    time.sleep(5)

    motorDataFrame = pd.read_html(driver.page_source)[2]
    motorDataFrame.rename(columns=motorDataFrame.iloc[1], inplace =True)
    motorDataFrame = motorDataFrame.iloc[2:]
    motorDataFrame = motorDataFrame[~motorDataFrame['월별'].isnull()] # NaN제거

    # 월의 자릿수에 따라 한자리인 경우 앞에 '0'을 붙임
    def assignMonth(row): 
        if int(row) >= 10: 
            result = row 
        else: 
            result = '0' + row 
        return result

    motorDataFrame['STD_YM'] = motorDataFrame['년도'].str[0:-1:1].copy() + motorDataFrame['월별'].str[0:-1:1].copy().apply(assignMonth) # str[start:stop:step])

    # empty Dataframe을 만들고(자동차 생산수량_월) 데이터 채움
    dfMOTOR_PROD_QUAN = pd.DataFrame(columns=['STD_YMD', 'MACRO_GBN', 'MACRO_CODE', 'MACRO_NM', 'MACRO_VALUE'])
    dfMOTOR_PROD_QUAN[['STD_YMD', 'MACRO_VALUE']] = motorDataFrame[['STD_YM', '생산']]
    dfMOTOR_PROD_QUAN['MACRO_GBN'] = 'D'
    dfMOTOR_PROD_QUAN['MACRO_CODE'] = '0106'
    dfMOTOR_PROD_QUAN['MACRO_NM'] = '자동차 생산수량_월'
    #display(dfMOTOR_PROD_QUAN)
    
    # empty Dataframe을 만들고(자동차 내수판매수량_월) 데이터 채움
    dfMOTOR_DOME_QUAN = pd.DataFrame(columns=['STD_YMD', 'MACRO_GBN', 'MACRO_CODE', 'MACRO_NM', 'MACRO_VALUE'])
    dfMOTOR_DOME_QUAN[['STD_YMD', 'MACRO_VALUE']] = motorDataFrame[['STD_YM', '내수']]
    dfMOTOR_DOME_QUAN['MACRO_GBN'] = 'D'
    dfMOTOR_DOME_QUAN['MACRO_CODE'] = '0107'
    dfMOTOR_DOME_QUAN['MACRO_NM'] = '자동차 생산수량_월'
    #display(dfMOTOR_DOME_QUAN)
    
    # empty Dataframe을 만들고(자동차 수출수량_월) 데이터 채움
    dfMOTOR_EXPORT_QUAN = pd.DataFrame(columns=['STD_YMD', 'MACRO_GBN', 'MACRO_CODE', 'MACRO_NM', 'MACRO_VALUE'])
    dfMOTOR_EXPORT_QUAN[['STD_YMD', 'MACRO_VALUE']] = motorDataFrame[['STD_YM', '수출(수량)']]
    dfMOTOR_EXPORT_QUAN['MACRO_GBN'] = 'D'
    dfMOTOR_EXPORT_QUAN['MACRO_CODE'] = '0108'
    dfMOTOR_EXPORT_QUAN['MACRO_NM'] = '자동차 수출수량_월'
    #display(dfMOTOR_EXPORT_QUAN)
    
    # empty Dataframe을 만들고(자동차 수출금액_월) 데이터 채움
    dfMOTOR_EXPORT_SALES = pd.DataFrame(columns=['STD_YMD', 'MACRO_GBN', 'MACRO_CODE', 'MACRO_NM', 'MACRO_VALUE'])
    dfMOTOR_EXPORT_SALES[['STD_YMD', 'MACRO_VALUE']] = motorDataFrame[['STD_YM', '수출(금액)']]
    dfMOTOR_EXPORT_SALES['MACRO_GBN'] = 'D'
    dfMOTOR_EXPORT_SALES['MACRO_CODE'] = '0109'
    dfMOTOR_EXPORT_SALES['MACRO_NM'] = '자동차 수출금액_월'
    #display(MOTOR_EXPORT_SALES)

    motorIT_EFAS_MACRO_RAW = pd.concat([dfMOTOR_PROD_QUAN, dfMOTOR_DOME_QUAN, dfMOTOR_EXPORT_QUAN, dfMOTOR_EXPORT_SALES])
    #display(motorIT_EFAS_MACRO_RAW)
        
    #motorIT_EFAS_MACRO_RAW.set_index('STD_YMD', inplace =True)
    motorIT_EFAS_MACRO_RAW.reset_index(drop=True)
    return motorIT_EFAS_MACRO_RAW


driver = webdriver.Chrome()
motorIT_EFAS_MACRO_RAW = motorCrawlData(driver)
motorIT_EFAS_MACRO_RAW.to_excel(f'{today}_motorIT_EFAS_MACRO_RAW.xlsx')
display(motorIT_EFAS_MACRO_RAW)
driver.close()  # 브라우저 닫기

['C67BD1D54EE8E36DB2280297D7A538B7', 'F58308E82FFDB14F0D511F0688321B83', 'DD86111EC9CECE909900779057FAD12C']


,STD_YMD,MACRO_GBN,MACRO_CODE,CLASS_NM,MACRO_NM,MACRO_VALUE
2,201001,D,0106,산업생산,자동차 생산수량_월,312847
3,201002,D,0106,산업생산,자동차 생산수량_월,279368
4,201003,D,0106,산업생산,자동차 생산수량_월,382173
5,201004,D,0106,산업생산,자동차 생산수량_월,381959
6,201005,D,0106,산업생산,자동차 생산수량_월,351819
...,...,...,...,...,...,...
178,202308,D,0109,산업생산,자동차 수출금액_월,4787539
179,202309,D,0109,산업생산,자동차 수출금액_월,4733577
180,202310,D,0109,산업생산,자동차 수출금액_월,5374104
181,202311,D,0109,산업생산,자동차 수출금액_월,6025934


In [27]:
#####################################################################
### 해운산업 (출처 : https://www.tradlinx.com/)
### HAEWOON_SCFI : 해운 운임지수-SCFI(컨테이너선)
### HAEWOON_CCFI : 해운 운임지수-SCFI(컨테이너선)
#####################################################################

today = date.today()

def haewoonMakeURL(driver):
    ## (TradLinx) 컨테이너 운임지표 : CCFI, SCFI
    # Data reference : Go to TradLinx homepage
    url = "https://www.tradlinx.com/freight-index" 
    driver.get(url)
    
    # Select Month because we need monthly data
    driver.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[3]/drp-input/div/div/span[2]').click()
    driver.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[3]/drp-input/div/div[2]/ul/li[2]').click() 

    print(driver.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[3]/drp-input/div/div/span[2]').text)

    driver.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[4]/div/i').click() # Pick calendar
    time.sleep(10)

    # Set query condition : Select start day as '2020.1.1'
    # Click 'JAN'
    while driver.find_element(By.CLASS_NAME, 'dtp-actual-month').text != 'JAN':
        driver.find_element(By.CLASS_NAME, "dtp-select-month-before").click()
    # Click '2010'
    while driver.find_element(By.CLASS_NAME, 'dtp-actual-year').text != '2010':
        driver.find_element(By.CLASS_NAME, "dtp-select-year-before").click()
    # Click 'day 1'
    driver.find_element(By.XPATH, "/html/body/div[2]/div/div[1]/div[3]/div[1]/table/tbody/tr[2]/td[6]/a").click()

    # Make sure that the selected day is 2010. JAN. 1
    print(driver.find_element(By.CLASS_NAME, 'dtp-actual-year').text)
    print(driver.find_element(By.CLASS_NAME, 'dtp-actual-month').text)
    print(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[1]/div[3]/div[1]/table/tbody/tr[2]/td[6]/a").text)
    print(driver.find_element(By.CLASS_NAME, 'dtp-btn-ok').text)

    driver.find_element(By.CLASS_NAME, 'dtp-btn-ok').click()
    time.sleep(5)
    #print(browser.find_element(By.XPATH, "/html/body").text)
    containerDataFrame = pd.read_html(driver.page_source)[1]

    containerDataFrame = containerDataFrame.transpose()
    containerDataFrame.columns = containerDataFrame.iloc[0]
    containerDataFrame = containerDataFrame[1:]
    containerDataFrame['STD_YM'] = containerDataFrame.index.str.replace(" ", "")

    containerDataFrame.columns.name = None  # Delete column name 'Date All'
    containerDataFrame.reset_index(drop=True, inplace=True) # index reset
    
    # empty Dataframe을 만들고(SCFI) 데이터 채움
    scfiIT_EFAS_MACRO_RAW = pd.DataFrame(columns=['STD_YMD', 'MACRO_GBN', 'MACRO_CODE', 'MACRO_NM', 'MACRO_VALUE'])
    scfiIT_EFAS_MACRO_RAW[['STD_YMD', 'MACRO_VALUE']] = containerDataFrame[['STD_YM', 'SCFI']]
    scfiIT_EFAS_MACRO_RAW['MACRO_GBN'] = 'I'
    scfiIT_EFAS_MACRO_RAW['MACRO_CODE'] = '0028'
    scfiIT_EFAS_MACRO_RAW['MACRO_NM'] = '글로벌_해운운임지수_SCFI(컨테이너선)_월'
    
    return scfiIT_EFAS_MACRO_RAW

driver = webdriver.Chrome()
scfiIT_EFAS_MACRO_RAW = haewoonMakeURL(driver)
display(scfiIT_EFAS_MACRO_RAW)
scfiIT_EFAS_MACRO_RAW.to_excel(f'{today}_scfiIT_EFAS_MACRO_RAW.xlsx')
driver.close()

Month
2010
JAN
01
OK


,STD_YMD,MACRO_GBN,MACRO_CODE,CLASS_NM,MACRO_NM,MACRO_VALUE
0,201301,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,1214.7
1,201302,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,1174.78
2,201303,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,1148.56
3,201304,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,1098.38
4,201305,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,1006.41
...,...,...,...,...,...,...
128,202309,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,956.03
129,202310,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,940.6
130,202311,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,1022.81
131,202312,I,0028,산업생산,글로벌_해운운임지수_SCFI(컨테이너선)_월,1230.22
